In [1]:
print("ok")

ok


In [5]:
import os
os.chdir('Data')

In [3]:
%pwd

'f:\\AI'

In [13]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
# extarct data from pdf files
def extract_pdf_data(Data):
    loader = DirectoryLoader(Data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [29]:
file_path = '../Data'
extract_data = extract_pdf_data(Data=file_path)

In [36]:
extract_data
len(extract_data)

24

In [41]:
# create chunks from the extracted data
def create_chunks(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=10)
    text_chunks = splitter.split_documents(data)
    return text_chunks

In [42]:
allCunks=create_chunks(extract_data)
len(allCunks)

24

In [46]:
#model embaded from huggingface
from langchain.embeddings import HuggingFaceEmbeddings
def download_model():
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}
    hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
    return hf

In [50]:
download_model()

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

In [83]:
query_result = download_model().embed_query(allCunks)

AttributeError: 'list' object has no attribute 'replace'

In [54]:
print(len(query_result))


384


In [57]:
#load env
from dotenv import load_dotenv
import os
load_dotenv()

True

In [74]:
PINECONE_API_KEY = os.environ.get("PINE_CONE_API_KEY")
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY # set the environment variable globally

In [70]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

# create index
def create_index(index_name):
    name = index_name
    dimension = 384
    metric="cosine"
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
    pc.create_index(name=name, dimension=dimension, metric=metric, spec=spec)
    return name
    

In [ ]:

# create index if not exists
try:
    createIndex= create_index('cookbot')
    print("Index created", createIndex)
except Exception as e:
    print("Index already exists")



Index already exists


In [82]:
from langchain_pinecone import PineconeVectorStore
print("Module installed successfully!")

 

ModuleNotFoundError: No module named 'langchain_pinecone'